# Reading and writing from Azure Cosmos DB

**In this lesson you:**
- Write data into Azure Cosmos DB
- Read data from Azure Cosmos DB

## Library Requirements

1. the Maven library with coordinate `com.databricks.training:databricks-cosmosdb-spark2.2.0-scala2.11:1.0.0` in the `https://files.training.databricks.com/repo` repository.
   - this allows a Databricks `spark` session to communicate with Azure Cosmos DB

The next cell walks you through installing the Maven library.

## Lab Setup

If you are running in an Azure Databricks environment that is already pre-configured with the libraries you need, you can skip to the next cell. To use this notebook in your own Databricks environment, you will need to create libraries, using the [Create Library](https://docs.azuredatabricks.net/user-guide/libraries.html) interface in Azure Databricks. Follow the steps below to attach the `azure-cosmosdb-spark` library to your cluster:

%md
1. Right click on the browser tab and select "Duplicate" to open a new tab.
1. In the left-hand navigation menu of your Databricks workspace, select **Clusters**, then select your cluster in the list. If it's not running, start it now.

  ![Select cluster](https://databricksdemostore.blob.core.windows.net/images/10-de-learning-path/select-cluster.png)

1. Select the **Libraries** tab (1), then select **Install New** (2). In the Install Library dialog, select **Maven** under Library Source (3). Under Coordinates, paste `com.databricks.training:databricks-cosmosdb-spark2.2.0-scala2.11:1.0.0` (4). Under Repository, paste `https://files.training.databricks.com/repo` (5), then select **Install** (6).
  
  ![Databricks new Maven library](https://databricksdemostore.blob.core.windows.net/images/14-de-learning-path/install-cosmosdb-spark-library.png)

1. Wait until the library successfully installs before continuing.

Once complete, return to this notebook to continue with the lesson.

##![Spark Logo Tiny](https://files.training.databricks.com/images/wiki-book/general/logo_spark_tiny.png) Load Azure Cosmos DB

Now load a small amount of data into Azure Cosmos DB to demonstrate that connection.

In [0]:
%run ./Includes/Classroom-Setup

Initialized classroom variables & functions...

Mounted datasets to /mnt/training from wasbs://training@dbtrainnortheurope.blob.core.windows.net/

Enter your Azure Cosmos DB account information in the cell below. Be sure to replace the **"cosmos-db-uri"** and **"your-cosmos-db-key"** values with your own before executing.

In [0]:
URI = "https://4v6j7i74extrg.documents.azure.com:443/"
PrimaryKey = "sSYFdpOje9fciSKbSD80vGRTX7LT5SHziR9kqeQsVw2eJtqiwqMH3XZ4QonZOtOtlB9yWmLCqwZNHK4AMrMSzg=="

All done!

<span>1.</span> Enter the Azure Cosmos DB connection information into the cell below. <br>

In [0]:
CosmosDatabase = "AdventureWorks"
CosmosCollection = "ratings"

cosmosConfig = {
  "Endpoint": URI,
  "Masterkey": PrimaryKey,
  "Database": CosmosDatabase,
  "Collection": CosmosCollection,
  "Upsert": "false"
}

<span>2.</span> Read the input parquet file.

In [0]:
from pyspark.sql.functions import col
ratingsDF = (spark.read
  .parquet("dbfs:/mnt/training/initech/ratings/ratings.parquet/")
  .withColumn("rating", col("rating").cast("double")))
print("Num Rows: {}".format(ratingsDF.count()))

Num Rows: 886782

In [0]:
display(ratingsDF)

product_id,user_id,rating
2,1,3.5
29,1,3.5
32,1,3.5
31,1,3.5
29,1,4.0
3,2,4.0
1,3,4.0
24,3,3.0
32,3,4.0
31,3,5.0


<span>3.</span> Write the data to Azure Cosmos DB.

In [0]:
ratingsSampleDF = ratingsDF.sample(.0001)

(ratingsSampleDF.write
  .mode("overwrite")
  .format("com.microsoft.azure.cosmosdb.spark")
  .options(**cosmosConfig)
  .save())

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-897507754379257> in <module> 
 4 . mode ( "overwrite" ) 
 5 . format ( "com.microsoft.azure.cosmosdb.spark" ) 
 ----> 6 . options ( ** cosmosConfig ) 
 7 .save())

 /databricks/spark/python/pyspark/sql/readwriter.py in save (self, path, format, mode, partitionBy, **options) 
 826 self . format ( format ) 
 827 if path is None : 
 --> 828 self . _jwrite . save ( ) 
 829 else : 
 830 self . _jwrite . save ( path ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 125 def deco ( * a , ** kw ) : 
 126 try : 
 --> 127 return f ( * a , ** kw ) 
 128 except py4j . protocol . Py4JJavaError as e : 
 129 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o481.save.
: java.lang.NoSuchMethodError: scala.Predef$.refArrayOps([Ljava/lang/Object;)Lscala/collection/mutable/ArrayOps;
	at com.microsoft.azure.cosmosdb.spark.config.Config$.getOptionsFromConf(Config.scala:278)
	at com.microsoft.azure.cosmosdb.spark.config.Config$.apply(Config.scala:226)
	at com.microsoft.azure.cosmosdb.spark.DefaultSource.createRelation(DefaultSource.scala:64)
	at org.apache.spark.sql.execution.datasources.SaveIntoDataSourceCommand.run(SaveIntoDataSourceCommand.scala:48)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:70)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult(commands.scala:68)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.doExecute(commands.scala:91)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:200)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$3(SparkPlan.scala:252)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:165)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:248)
	at org.apache.spark.sql.execution.SparkPlan.execute(SparkPlan.scala:192)
	at org.apache.spark.sql.execution.QueryExecution.toRdd$lzycompute(QueryExecution.scala:158)
	at org.apache.spark.sql.execution.QueryExecution.toRdd(QueryExecution.scala:157)
	at org.apache.spark.sql.DataFrameWriter.$anonfun$runCommand$1(DataFrameWriter.scala:1018)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withCustomExecutionEnv$5(SQLExecution.scala:116)
	at org.apache.spark.sql.execution.SQLExecution$.withSQLConfPropagated(SQLExecution.scala:248)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withCustomExecutionEnv$1(SQLExecution.scala:101)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:841)
	at org.apache.spark.sql.execution.SQLExecution$.withCustomExecutionEnv(SQLExecution.scala:77)
	at org.apache.spark.sql.execution.SQLExecution$.withNewExecutionId(SQLExecution.scala:198)
	at org.apache.spark.sql.DataFrameWriter.runCommand(DataFrameWriter.scala:1018)
	at org.apache.spark.sql.DataFrameWriter.saveToV1Source(DataFrameWriter.scala:439)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:423)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.refl

<span>4.</span> Confirm that your data is now in Azure Cosmos DB.

In [0]:
dfCosmos = (spark.read
  .format("com.microsoft.azure.cosmosdb.spark")
  .options(**cosmosConfig)
  .load())

display(dfCosmos)
